In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [98.9 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [992 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-04 23:30:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-11-04 23:30:09 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GamesETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
# Read in data from GCS Buckets
from pyspark import SparkFiles
url="https://storage.googleapis.com/big-data-bowl/games.csv"
spark.sparkContext.addFile(url)
games_data_df = spark.read.csv(SparkFiles.get("games.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
games_data_df.show()

+----------+------+----+----------+---------------+------------+---------------+
|    gameId|season|week|  gameDate|gameTimeEastern|homeTeamAbbr|visitorTeamAbbr|
+----------+------+----+----------+---------------+------------+---------------+
|2021090900|  2021|   1|09/09/2021|       20:20:00|          TB|            DAL|
|2021091200|  2021|   1|09/12/2021|       13:00:00|         ATL|            PHI|
|2021091201|  2021|   1|09/12/2021|       13:00:00|         BUF|            PIT|
|2021091202|  2021|   1|09/12/2021|       13:00:00|         CAR|            NYJ|
|2021091203|  2021|   1|09/12/2021|       13:00:00|         CIN|            MIN|
|2021091204|  2021|   1|09/12/2021|       13:00:00|         DET|             SF|
|2021091205|  2021|   1|09/12/2021|       13:00:00|         HOU|            JAX|
|2021091206|  2021|   1|09/12/2021|       13:00:00|         IND|            SEA|
|2021091207|  2021|   1|09/12/2021|       13:00:00|         TEN|            ARI|
|2021091208|  2021|   1|09/1

In [8]:
games_data_df.dtypes

[('gameId', 'int'),
 ('season', 'int'),
 ('week', 'int'),
 ('gameDate', 'string'),
 ('gameTimeEastern', 'string'),
 ('homeTeamAbbr', 'string'),
 ('visitorTeamAbbr', 'string')]

Postgres Setup

In [9]:
# Store environment variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for Cloud SQL
mode = "append"
jdbc_url="jdbc:postgresql://34.72.136.99:5432/big-data-bowl"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [10]:
# Write DataFrame to games table in Cloud SQL
games_data_df.write.jdbc(url=jdbc_url, table='games', mode=mode, properties=config)